In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
%run ../py/sql_utlis.py

su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()

Wall time: 11 s


In [31]:

sql_str = '''
    SELECT
        nps.[file_name_id],
        COUNT(nps.[file_name_id]) AS total_ids
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    GROUP BY [file_name_id]
    ORDER BY COUNT(nps.[file_name_id]) DESC;'''
counts_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
counts_df

,file_name_id,total_ids
0,516,1125
1,485,379
2,160,297
3,486,263
4,250,257
...,...,...
527,141,11
528,458,4
529,301,3
530,269,3


In [32]:

file_name_id = counts_df.head(1).file_name_id.squeeze()
sql_str = f'''
    SELECT
        fn.[file_name],
        np.[navigable_parent_id],
        nps.[sequence_order]
    FROM
        [Jobhunting].[dbo].[FileNames] fn INNER JOIN
        [Jobhunting].[dbo].[NavigableParentSequence] nps ON
        fn.[file_name_id] = nps.[file_name_id] INNER JOIN
        [Jobhunting].[dbo].[NavigableParents] np ON
        nps.[navigable_parent_id] = np.[navigable_parent_id]
    WHERE fn.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
sequence_df

,file_name,navigable_parent_id,sequence_order
0,Statistician_(Data_Scientist)_12_month_Roster_...,3104,0
1,Statistician_(Data_Scientist)_12_month_Roster_...,3119,1
2,Statistician_(Data_Scientist)_12_month_Roster_...,3462,2
3,Statistician_(Data_Scientist)_12_month_Roster_...,3459,3
4,Statistician_(Data_Scientist)_12_month_Roster_...,14268,4
...,...,...,...
1120,Statistician_(Data_Scientist)_12_month_Roster_...,2634,1121
1121,Statistician_(Data_Scientist)_12_month_Roster_...,15419,1122
1122,Statistician_(Data_Scientist)_12_month_Roster_...,2559,1123
1123,Statistician_(Data_Scientist)_12_month_Roster_...,2634,1124


In [14]:

def guess_seq_len(seq):
    guess = 1
    max_len = int(len(seq) / 2)
    for x in range(2, max_len):
        if seq[0:x] == seq[x:2*x]:
            
            return x
    
    return guess

In [28]:

sql_str = f'''
    SELECT nps.[navigable_parent_id]
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
cutoff = guess_seq_len(navigable_parent_ids_list)
while cutoff > 1:
    sql_str = f'''
        DELETE FROM [Jobhunting].[dbo].[NavigableParentSequence]
        WHERE
            [sequence_order] < {cutoff} AND
            [file_name_id] = {file_name_id};'''
    CURSOR.execute(sql_str)
    sql_str = f'''
        UPDATE [Jobhunting].[dbo].[NavigableParentSequence]
        SET [sequence_order] = [sequence_order] - {cutoff}
        WHERE [file_name_id] = {file_name_id};'''
    CURSOR.execute(sql_str)
    sql_str = f'''
        SELECT nps.[navigable_parent_id]
        FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
        WHERE nps.[file_name_id] = {file_name_id}
        ORDER BY nps.sequence_order;'''
    sequence_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
    navigable_parent_ids_list = sequence_df.navigable_parent_id.tolist()
    cutoff = guess_seq_len(navigable_parent_ids_list)

In [30]:

sql_str = '''
    DELETE FROM [Jobhunting].[dbo].[NavigableParentSequence]
    WHERE [navigable_parent_sequence_id] IN (
        SELECT h1.[navigable_parent_sequence_id]
        FROM
            [Jobhunting].[dbo].[NavigableParentSequence] AS h1 INNER JOIN
            [Jobhunting].[dbo].[NavigableParentSequence] AS h2 ON
                h1.[file_name_id] = h2.[file_name_id] AND
                h2.[navigable_parent_id] = h1.[navigable_parent_id] AND
                h2.[sequence_order] = h1.[sequence_order] - 1
        );'''
CURSOR.execute(sql_str)

In [34]:

sql_str = f'''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps
    WHERE nps.[file_name_id] = {file_name_id}
    ORDER BY nps.sequence_order;'''
sequence_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
sequence_df

,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
0,29628,516,3104,0,0
1,29629,516,3119,1,0
2,29630,516,3462,2,0
3,29631,516,3459,3,0
4,29632,516,14268,4,0
...,...,...,...,...,...
1120,30749,516,2634,1121,0
1121,30750,516,15419,1122,0
1122,30751,516,2559,1123,0
1123,30752,516,2634,1124,0


In [35]:

mask_series = sequence_df.duplicated(subset='navigable_parent_id', keep=False)
sequence_df[mask_series]

,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
1,29629,516,3119,1,0
2,29630,516,3462,2,0
3,29631,516,3459,3,0
4,29632,516,14268,4,0
5,29633,516,14270,5,0
...,...,...,...,...,...
1120,30749,516,2634,1121,0
1121,30750,516,15419,1122,0
1122,30751,516,2559,1123,0
1123,30752,516,2634,1124,0


In [40]:

# Get a list of html strings that repeat somewhere in the posting
navigable_parent_ids_list = sequence_df.navigable_parent_id.unique().tolist()
for navigable_parent_id in navigable_parent_ids_list:
    
    # Get their sequence numbers
    mask_series = (sequence_df.navigable_parent_id == navigable_parent_id)
    sequence_orders_list = sequence_df[mask_series].sequence_order.tolist()
    
    # Get the next html strings
    next_tags_list = []
    for sequence_order in sequence_orders_list:
        mask_series = (sequence_df.sequence_order == sequence_order+1)
        next_tag = sequence_df[mask_series].navigable_parent_id.squeeze()
        if type(next_tag) != pd.Series:
            next_tags_list.append(next_tag)
    
    # Check for sequence duplication
    if len(next_tags_list) != len(set(next_tags_list)):
        print(navigable_parent_id, next_tags_list)

3119 [3462, 3462, 3462]
3462 [3459, 3459, 3459, 3459, 3459]
3459 [14268, 14268, 14268, 14268, 14268]
14268 [14270, 14270, 14270, 14270, 14270, 14270]
14270 [14258, 14258, 14258, 14258, 14258, 14258]
14258 [14266, 14266, 14266, 14266, 14266, 14266]
14266 [14254, 14254, 14254, 14254, 14254, 14254]
14254 [14249, 14249, 14249, 14249, 14249, 14249]
14249 [3119, 3119, 3462, 3462, 14268, 3117]
3117 [2564, 2564, 2564, 2564]
2564 [14246, 14246, 14246, 14246, 2562]
14246 [2562, 2562, 2562, 2564]
2562 [2801, 2801, 2801, 2801, 2801]
2801 [2901, 2901, 2901, 2901, 2901]
2901 [3116, 3117, 3117, 2562, 832]
832 [1758, 1758, 1758]
1758 [15616, 15616, 15616]
15616 [309, 309, 309, 309]
309 [13306, 537, 609, 14363, 13306, 537, 609, 14363, 13306, 13306, 537, 537, 537, 609, 14363, 609, 14363]
13306 [1640, 1640, 1640, 2077]
1640 [2077, 2077, 2077]
2077 [15585, 15585, 15585, 15585]
15585 [2080, 2080, 2080, 2080]
2080 [15594, 15594, 15594, 15594]
15594 [505, 505, 505, 505]
505 [10133, 10133, 10133, 10133]
10133

In [41]:

from suffix_trees import STree

# Suffix-Tree example.
st = STree.STree('abcdefghab')
print(st.find('abc')) # 0
print(st.find_all('ab')) # [0, 8]

# Generalized Suffix-Tree example.
a = ['xxxabcxxx', 'adsaabc', 'ytysabcrew', 'qqqabcqw', 'aaabc']
st = STree.STree(a)
print(st.lcs()) # 'abc'

0
{0, 8}
abc


In [43]:

[f'STree.{fn}' for fn in dir(STree) if not fn.startswith('_')]

['STree.STree']

In [44]:

STree.STree?

Init signature: STree.STree(input='')
Docstring:      Class representing the suffix tree.
File:           c:\users\dev\documents\repositories\job-hunting\jh_env\lib\site-packages\suffix_trees\stree.py
Type:           type
Subclasses:     


In [47]:

id = 5419
str(id).zfill(6)

'005419'